In [0]:
from pyspark.sql.functions import col, monotonically_increasing_id
from delta.tables import DeltaTable

# CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

# CREATING DIMENSION MODEL

### Fetch Relative Column

In [0]:
df_scr = spark.sql('''
SELECT DISTINCT(Branch_ID) as Branch_ID , BranchName
FROM PARQUET.`abfss://silver@storageact04.dfs.core.windows.net/carsales`
'''
)

In [0]:
df_scr.display()

Branch_ID,BranchName
BR0131,Audi Motors
BR0760,Healey Motors
BR0789,Hillman Motors
BR0938,Isotta Fraschini Motors
BR1040,Lada Motors
BR1693,Saleen Motors
BR1792,Simca do Brasil Motors
BR1799,Simca do Brasil Motors
BR1955,Toyota Motors
BR1978,Turner Motors


### dim_model Sink - Initial and Incremental(just bring the schema if table NOT EXISTS)

In [0]:
if spark.catalog.tableExists('car_catalog.gold.dim_branch'):
  df_sink = spark.sql('''
    SELECT Dim_branch_key, Branch_ID , BranchName
    FROM car_catalog.gold.dim_branch
    '''
    )  
else:
    df_sink = spark.sql('''
    SELECT 1 AS Dim_branch_key, Branch_ID , BranchName
    FROM PARQUET.`abfss://silver@storageact04.dfs.core.windows.net/carsales`
    WHERE 1=0
    '''
    )

### Filtering new records and Updating old records

In [0]:
df_filter = df_scr.join(df_sink, df_scr['Branch_ID'] == df_sink['Branch_ID'], 'left').select(df_scr['Branch_ID'],df_scr['BranchName'],df_sink['Dim_branch_key'])

In [0]:
df_filter.display()

Branch_ID,BranchName,Dim_branch_key
BR0131,Audi Motors,null
BR0760,Healey Motors,null
BR0789,Hillman Motors,null
BR0938,Isotta Fraschini Motors,null
BR1040,Lada Motors,null
BR1693,Saleen Motors,null
BR1792,Simca do Brasil Motors,null
BR1799,Simca do Brasil Motors,null
BR1955,Toyota Motors,null
BR1978,Turner Motors,null


**DF Filter Old**

In [0]:
df_filter_old = df_filter.filter(col('dim_branch_key').isNotNull())

In [0]:
df_filter_old.display()

Branch_ID,BranchName,Dim_branch_key


**DF_Filter_New**

In [0]:
df_filter_new = df_filter.filter(col('dim_branch_key').isNull()).select(df_filter['Branch_ID'],df_filter['BranchName'])

In [0]:
df_filter_new.display()

Branch_ID,BranchName
BR0131,Audi Motors
BR0760,Healey Motors
BR0789,Hillman Motors
BR0938,Isotta Fraschini Motors
BR1040,Lada Motors
BR1693,Saleen Motors
BR1792,Simca do Brasil Motors
BR1799,Simca do Brasil Motors
BR1955,Toyota Motors
BR1978,Turner Motors


### Create Surrogate Key 

**Fetch the max Surrogate Key**

In [0]:
if incremental_flag ==  '0' :
    max_value = 1
else:
    max_value_df = spark.sql("SELECT max(Dim_branch_key) FROM cars_catalog.gold.dim_branch")
    max_value = max_value_df.collect()[0][0]+1

**Create Surrogate Key column and Add the max Surrogate key**

In [0]:
max_value

1

In [0]:
df_filter_new = df_filter_new.withColumn('Dim_branch_key', max_value + monotonically_increasing_id())

### Create Final Filter - df_filter_old + df_filter_new

In [0]:
df_final = df_filter_new.union(df_filter_old)

In [0]:
df_final.display()

Branch_ID,BranchName,Dim_branch_key
BR0131,Audi Motors,1
BR0760,Healey Motors,2
BR0789,Hillman Motors,3
BR0938,Isotta Fraschini Motors,4
BR1040,Lada Motors,5
BR1693,Saleen Motors,6
BR1792,Simca do Brasil Motors,7
BR1799,Simca do Brasil Motors,8
BR1955,Toyota Motors,9
BR1978,Turner Motors,10


# SCD TYPE - 1 (UPSERT)

In [0]:
#Incremental Run 
if spark.catalog.tableExists('car_catalog.gold.dim_branch'):
    delta_tbl = DeltaTable.forPath(spark, 'abfss://gold@storageact04.dfs.core.windows.net/dim_branch')
    delta_tbl.alias('trg').merge(df_final.alias('src'), 'trg.dim_branch_key = src.Dim_branch_key')\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                            .execute()
    
#Initial run
else:
    df_final.write.format('delta')\
    .mode('overwrite')\
    .option('path','abfss://gold@storageact04.dfs.core.windows.net/dim_branch')\
    .saveAsTable('cars_catalog.gold.dim_branch')

In [0]:
%sql
SELECT * FROM cars_catalog.gold.dim_branch

Branch_ID,BranchName,Dim_branch_key
BR0131,Audi Motors,1
BR0760,Healey Motors,2
BR0789,Hillman Motors,3
BR0938,Isotta Fraschini Motors,4
BR1040,Lada Motors,5
BR1693,Saleen Motors,6
BR1792,Simca do Brasil Motors,7
BR1799,Simca do Brasil Motors,8
BR1955,Toyota Motors,9
BR1978,Turner Motors,10
